---
title: How to submit a PwBaseWorkChain 
subtitle: 
#author:
#  - name: Miki Bonacci
    #affiliations: Executable Books; Curvenote
    #orcid: 0000-0002-7859-8394
#    email: miki.bonacci@psi.ch
license:
  code: MIT
#date: 2023/01/23
---

In the following we show how to run a calculation with the `aiida-quantumespresso` plugin using the new *atomistic* `StructureData`. 
The procedure is very similar to the [old way](https://aiida-quantumespresso.readthedocs.io/en/latest/tutorials/first_pw.html), except to the fact that now properties like magnetization and charge are provided within the structure
and not in the input parameters of the `PwCalculation`.

***For now, only magnetization is supported.***

***TO MAKE IT WORK, PLEASE USE THE FOLLOWING VERSIONS OF THE RELATED CODES:***

- aiida-core: [new_StructureData branch](https://github.com/mikibonacci/aiida-core/tree/new_StructureData)
- aiida-quantumespresso: [new_StructureData branch](https://github.com/mikibonacci/aiida-quantumespresso/tree/new_StructureData)
- aiida-pseudo: [new_StructureData branch](https://github.com/mikibonacci/aiida-pseudo/tree/new_StructureData)

In [1]:
from aiida import load_profile, orm
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_atomistic import StructureData, StructureDataMutable

load_profile()

Profile<uuid='1a5a8d0836814a04a238c67cc7481655' name='default'>

Once loaded the necessary modules, we start building up our `StructureDataMutable`, to be then converted into the AiiDA `StructureData` (via the `to_immutable` method):

In [2]:
mutable_structure = StructureDataMutable()
mutable_structure.set_cell([[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]])

mutable_structure.add_atom({
            'symbol':'Si',
            'position':[1/2, 1/2, 1/2],
            'kind_name': 'Si1'
        })
mutable_structure.add_atom({
            'symbol':'Si',
            'position':[3/4, 3/4, 3/4],
            'kind_name': 'Si1'
        })

structure = mutable_structure.to_immutable()

We then generate the `PwBaseWorkChain` builder via the `get_builder_from_protocol` method, providing as input the *pw.x* code node and the structure:

In [3]:
from aiida.engine import submit, run_get_node


builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=structure,
    protocol="fast",
    )

run = run_get_node(builder)

09/03/2024 09:41:36 AM <533575> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1297|PwBaseWorkChain|run_process]: launching PwCalculation<1302> iteration #1
09/03/2024 09:41:40 AM <533575> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1297|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/03/2024 09:41:40 AM <533575> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1297|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


After the job is finished, you can inspect the outputs as usual:

In [4]:
run[0]

{'remote_folder': <RemoteData: uuid: bd399913-0e71-447a-8c53-6287970ebd70 (pk: 1303)>,
 'retrieved': <FolderData: uuid: b7264589-2311-4dad-aeb4-e270acc7a02b (pk: 1304)>,
 'output_parameters': <Dict: uuid: c46461bc-469c-461b-ade5-21bcc95c8890 (pk: 1307)>,
 'output_trajectory': <TrajectoryData: uuid: d2f366b0-ded0-4f2b-a3a8-e7b1f26798c8 (pk: 1306)>,
 'output_band': <BandsData: uuid: 79d9187c-b3ca-4c31-85b0-2c79cb145a37 (pk: 1305)>}

## How to deal with magnetic configurations

The first way to prepare a magnetic calculation is to trigger the `nspin` or `noncolin` in the input parameters of the `PwCalculation` respectively for *collinear* (`nspin=2`) and *non collinear* (`noncolin=.True.`) calculations.

Of course, the magnetic moments have to be provided within the `StructureData`, so that they can be used to build the [*starting_magnetization*](https://www.quantum-espresso.org/Doc/INPUT_PW.html#idm301) variable of the `&SYSTEM` namelist.


### Collinear case 

We first load the structure using Pymatgen to parse the *mcif* file for BCC Iron:

In [5]:
from pymatgen.core import Structure

#suppose you are in the `aiida-atomistic` root folder
iron_bcc = Structure.from_file('../examples/structure/data/Fe_bcc.mcif', primitive=True)
magnetic_structure = StructureData.from_pymatgen(iron_bcc)

print(magnetic_structure.properties.magmoms)

[[0.0, 0.0, 2.5]]


In [6]:
magnetic_structure.properties.kinds

['Fe1']

We can check that our structure is collinear (trivial in this case):

In [7]:
magnetic_structure.is_collinear

True

We can create again the builder as before, this time overriding `nspin` to be equal to 2 (*collinear* calculation):

In [8]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"nspin": 2}
            }
        }
    }
    )

In [9]:
run = run_get_node(builder)

09/03/2024 09:41:42 AM <533575> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1314|PwBaseWorkChain|run_process]: launching PwCalculation<1319> iteration #1
09/03/2024 09:41:51 AM <533575> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1314|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/03/2024 09:41:51 AM <533575> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1314|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


### The output magnetic moments

For now, the new output magnetization of a structure will not be stored in the output `StructureData` (if any). 
The reason is that the output magnetization can present some discrepancies which are actually artifacts of the simulation. E.g., symmetries maybe not entirely used (due to different kinds) and so magnetization magnitudes (and orientations) are not the same as expected. 

So, the if the user needs it, he can use a `calcfuntion` to generate a new `StructureData` with the new magnetic moments (both as list of *floats* or of len=3 *lists*):

In [10]:
trajectory = run[-1].called[-1].outputs.output_trajectory # run[1].called[-1].outputs.output_trajectory
magnetic_moments = [magmom[0] for magmom in trajectory.get_array('atomic_magnetic_moments')]
print(magnetic_moments)

[2.3254]


In [11]:
from aiida_quantumespresso.utils.magnetic import generate_structure_with_magmoms

new_structure = generate_structure_with_magmoms(magnetic_structure, magnetic_moments)
print(f"New StructureData: {new_structure}\nAttached magmoms: {new_structure.properties.magmoms}")

New StructureData: uuid: ccd7f6ab-22d6-4677-9294-2f2e51973c47 (pk: 1327)
Attached magmoms: [[0.0, 0.0, 2.3254]]


If you don't want the provenance and the new `StructureData` node to be stored, add `metadata={"store_provenance": False}` to the input of the `calcfunction`. 

### Non-collinear case 

For the non-collinear case, we just need to put `noncolin: True` in the input parameters:

In [12]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"noncolin": True}
            }
        }
    }
    )

In [13]:
run = run_get_node(builder)

09/03/2024 09:41:53 AM <533575> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1333|PwBaseWorkChain|run_process]: launching PwCalculation<1338> iteration #1
09/03/2024 09:42:10 AM <533575> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1333|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/03/2024 09:42:11 AM <533575> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1333|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


In [14]:
run[0]

{'remote_folder': <RemoteData: uuid: 817fef82-67a1-4878-8433-a7659101a43e (pk: 1339)>,
 'retrieved': <FolderData: uuid: 820be53b-bfb6-49df-937f-b27ae287becf (pk: 1340)>,
 'output_parameters': <Dict: uuid: 4d431f19-349d-4d6b-8b4a-878a43976647 (pk: 1343)>,
 'output_trajectory': <TrajectoryData: uuid: d72b6842-aeea-4b5d-96bb-514aa93424ae (pk: 1342)>,
 'output_band': <BandsData: uuid: f75ce733-cd14-416a-b2e7-3f9b669e43e5 (pk: 1341)>}

Currently, no parser for non-collinear magnetization is implemented in the `aiida-quantumespresso` plugin.